# Radiospecta

* Sunpy provisionally affiliated package.
* Provides `Fido` clients for a number of space and ground based observatories or observatory networks
  * RSTN, EOVSA, WIND/WAVES, STEREO WAVES, Parker Solar Probe/RFS, CALLISTO
* Provides a `Spectrogram` object or container like `TimeSeries` or `Map` but for dynamic spectra or spectrograms
* Docs https://docs.sunpy.org/projects/radiospectra/en/latest/

### Today will cover
1. Search and download data
2. Loading data into spectrogram
3. Plotting individual spectra
4. Loading custom data into a Spectrogram
5. Make a combined plot from 1GhZ to 1kHz

## Imports

In [ ]:
from datetime import datetime

import astropy.units as u
import numpy as np

from astropy.time import Time
from astropy.io import fits
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

from sunpy.net import Fido, attrs as a

from radiospectra import net #let Fido know about the radio clients
from radiospectra.spectrogram2 import Spectrogram # in the process of updating old spectrogram

# What can be searched for?

In [ ]:
Fido

In [ ]:
a.Instrument.waves   #try .waves .swave .rstn .ecallisto

# Lets look for a lot of radio data from 2017-09-02 15:00 - 18:00
* Use `Fido` like yesterday but specifiy different instruments
* `|` can be used as `or` so instrument `a or b` is `a | b`
* Search for data from WIND/WAVES, RSTN, CALLISTO, and SWAVES
* Small bug the `callisto` client is called `ecallisto`

In [ ]:
query = Fido.search()

Print the results and see what was found

In [ ]:
query

# Many results let look at the RSTN data first

* First download the the data
* Same as yesterday can just pass the query to `Fido.fetch`

In [ ]:
rstn_files = Fido.fetch(query['rstn'])

The `Fido.fetch` call basially returns a lis of filepaths e.g.

In [ ]:
rstn_files

## Load into spectrograms 

* Very similar to `Map` and `TimeSeries` the `Spectrogram` can take a list of:
   * data header pairs
   * filepaths
   * urls
   * or a combinatino of the above
* Since fido returns a list of file paths we can just pass this directly to `Spectrogram`

In [ ]:
rstn_spec = Spectrogram(sorted(rstn_files))

Since we gave it a list of filepaths should get back a list of spectrograms

In [ ]:
type(rstn_spec), len(rstn_spec)

Finnally spectrogram hava short repr or text representation

In [ ]:
rstn_spec

Once the data is loaded into a spectrogram it can be easily plotted using the `plot` method and the plots
can be adjusted like a normal matplotlib plots 

In [ ]:
rstn_spec[0].plot()

In [ ]:
rstn_spec[1].plot()

There isn't an obvious radio busrt in the RSTN data so move on to WAVES

## Now do the same steps but for waves
* Remeber you can access the waves part of the query using `query['waves']`

Download WAVES

In [ ]:
# Code to download the waves do here

waves_files = 

Load into spectrogram

In [ ]:
# Load
waves_spec = 

What is the type and size of the result

In [ ]:
# Check the type and length here
type(waves_spec), len(waves_spec)

Print a string reprentation a WAVES spectrogmram

Plot the WAVES spectrogram

In [ ]:
# plot the first spectrogram e.g. waves_spec[0]


In [ ]:
# plot the first spectrogram e.g. waves_spec[0]


### Waves data looks like it captured a burst lets make a nicer figure

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)

waves_spec[0].plot(axes=axes[0])
waves_spec[1].plot(axes=axes[1])

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)

waves_spec[0].plot(axes=axes[0])
waves_spec[1].plot(axes=axes[1])

# by default y-axis low to hight flip so moving away fro sun with time
axes[0].set_ylim(reversed(axes[0].get_ylim()))
axes[1].set_ylim(reversed(axes[1].get_ylim()))


In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)

waves_spec[0].plot(axes=axes[0])
waves_spec[1].plot(axes=axes[1])

# by default y-axis low to hight flip so moving away fro sun with time
axes[0].set_ylim(reversed(axes[0].get_ylim()))
axes[1].set_ylim(reversed(axes[1].get_ylim()))

# log y-axis
axes[0].set_yscale('log')
axes[1].set_yscale('log')

In [ ]:
from datetime import datetime

fig, axes = plt.subplots(2, 1, sharex=True)

waves_spec[0].plot(axes=axes[0])
waves_spec[1].plot(axes=axes[1])

# by default y-axis low to hight flip so moving away fro sun with time
axes[0].set_ylim(reversed(axes[0].get_ylim()))
axes[1].set_ylim(reversed(axes[1].get_ylim()))

# log y-axis
axes[0].set_yscale('log')
axes[1].set_yscale('log')

axes[1].set_xlim(datetime(2017, 9, 2, 15), datetime(2017, 9, 2, 18))

# Find some callisto data?
* http://soleil.i4ds.ch/solarradio/callistoQuicklooks/?date=20170902
* Glasgow data looks good, what about others, Greenland?

We need to search within the `Fido.seach` results.

Remember the columns can be accessed using their names `query['CALLISTO']['Observatory']` and work very similar to numpy array so you can do `query['CALLISTO']['Observatory'] == 'my fave'`

In [ ]:
index = 
query['CALLISTO'][index]

In [ ]:
# Only download a sub set of the returned data
callisto_files = Fido.fetch(query['CALLISTO'][index], max_conn=1)
callisto_files = Fido.fetch(callisto_files)

Again load the data into the spectrogram

In [ ]:
callisto_specs = 
callisto_specs

In [ ]:
fig, axes = plt.subplots()
[sp.plot(axes=axes) for sp in callisto_specs]
# order not guarnteed so need to set plot range 
axes.set_xlim(datetime(2017, 9, 2, 15, 25), datetime(2017, 9, 2, 18, 10))

# Load data not supported by Spectrogram

In [ ]:
data = np.zeros((200, 200))
data[50:150, 50:150] = 1
times = Time('2017-09-02 15:00') + np.arange(200) * u.s
freqs = np.linspace(10, 20, 200) * u.GHz

# expects at least the following meta data
meta = {
    'observatory': 'some place',
    'instrument': 'an instrument',
    'detector': 'the detector',
    'freqs': freqs,
    'times': times,
    'wavelength': a.Wavelength(freqs[0], freqs[-1]),
    'start_time': times[0],
    'end_time': times[-1]
}
super_spec = Spectrogram(data, meta )

In [ ]:
super_spec

In [ ]:
super_spec.plot()

# Use the Orfees web site to find and dowload data
* Download data to you local machine
* https://rsdb.obs-nancay.fr/

In [ ]:
orfee = fits.open(<path goes here>)
orfee

In [ ]:
# Extract and stack the subbands for stokes I and V
orfee_i = np.hstack([orfee[2].data[f'STOKESI_B{i}'] for i in range(1, 6)]).T
orfee_v = np.hstack([orfee[2].data[f'STOKESV_B{i}'] for i in range(1, 6)]).T

orfee_time_str = orfee[0].header['DATE-OBS'] #+ 'T' + orfee[0].header['TIME-OBS']
# orfee_time_str = orfee_time_str[:-4] +'.' + orfee_time_str[-3:]

# Calculate time axis 
orfee_times = Time(orfee_time_str) + (orfee[2].data['TIME_B1']/1000)*u.s # times are not the same for all sub spectra!

# Calculate frequency axis by stacking the freqcies from each subband
orfee_freqs = np.hstack([orfee[1].data[f'FREQ_B{i}'] for i in range(1, 6)]) *u.MHz

In [ ]:
# Construct meta data dictionary
orfee_meta = {
    'observatory': orfee[0].header['ORIGIN'],
    'instrument': orfee[0].header['INSTRUME'],
    'detector': orfee[0].header['INSTRUME'],
    'freqs': orfee_freqs.reshape(-1),
    'times': orfee_times,
    'wavelength': a.Wavelength(orfee_freqs[0,0], orfee_freqs[0,-1]),
    'start_time': orfee_times[0],
    'end_time': orfee_times[-1]
}

# Load data in spectrogram
orfee_spec_i = Spectrogram(orfee_i, orfee_meta)
orfee_spec_v = Spectrogram(orfee_v, orfee_meta)
orfee_spec_i

In [ ]:
vmm = np.percentile(orfee_spec_i.data, [1,99])
orfee_spec_i.plot(norm=LogNorm(vmin=vmm[0], vmax=vmm[1]))


In [ ]:
vmm = np.percentile(orfee_spec_v.data, [1,99])
orfee_spec_v.plot(vmin=vmm[0], vmax=vmm[1])
plt.semilogy()
plt.ylim(plt.ylim()[::-1])

# Final combined plot 1 GHz to 1 kHz

In [ ]:
from datetime import datetime
from matplotlib.colors import LogNorm

# set up the plots 4 rows 1 column with shared common x-axis
fig, axes = plt.subplots(4, 1, sharex=True, figsize=(10, 10))

waves_spec[0].plot(axes=axes[0], norm=LogNorm())
waves_spec[1].plot(axes=axes[1], norm=LogNorm())

[sp.plot(axes=axes[2]) for sp in callisto_specs]

vmm = np.percentile(orfee_spec_v.data, [1,99])
orfee_spec_v.plot(axes=axes[3], vmin=vmm[0], vmax=vmm[1])

# by default y-axis low to high flip so moving away from sun with time
[axes[i].set_ylim(reversed(axes[i].get_ylim())) for i in range(4)]

# log y-axis
[axes[i].set_yscale('log') for i in range(4)]

# reduce time range
axes[1].set_xlim(datetime(2017, 9, 2, 15, 30), datetime(2017, 9, 2, 17))